In [1]:
import re
import sys
import os
import json
import requests
from bs4 import BeautifulSoup

# Initialization
Enter input files names. Input files should be named organ_name.tsv

In [3]:
file_name = "Large_Intestine.tsv"
read_file = open(file_name, "r")

## Extract ASCT+B Cell Values and store in a dictionary
excel_dict = keys are columns (CT/BGene) and values are rows <br>
version = version of the asct+b table <br>
key_list = column headers. ie (CT/BGene)

In [4]:
def tsv_to_columns_row_dictionary(read_file):
    excel_dict = dict()
    key_list = list()
    for count, row in enumerate(read_file):
        temp = re.split(r'\t', row.strip("\n"))
        if count == 8:
            version = temp[1]
        if count == 10:
            for value in temp:
                excel_dict[value] = list()
                key_list.append(value)       
        if count > 10:
            for count, value in enumerate(temp):
                excel_dict[key_list[count]].append(value)
    return(excel_dict, key_list, version)

# Output of column/row dictionary
excel_dict, key_list, version = tsv_to_columns_row_dictionary(read_file)


## Extract Cell Types of the Highest Resolution from Cell ID. 
Extract cell ID and match to cell name using ebi api <br>
If no cell ontology is found, cell type name on ASCT+B table is used
cell_type_list = list of cell types <br>
as_name = organ name

In [5]:
def get_jdata(url):
    myResponse = requests.get(url)
    if (myResponse.ok):
        j_data = json.loads(myResponse.content.decode())
        cell_type = j_data['_embedded']['terms'][0]['label']

        # Case where the cell ontology term is replaced 
        if "CL" in cell_type:
            cell_id = j_data['_embedded']['terms'][0]['term_replaced_by']
            url = "http://www.ebi.ac.uk/ols/api/terms?iri=http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2F{}".format(cell_id)
            cell_type = get_jdata(url)

    return cell_type

def add_cell_type_to_list(excel_dict, key_list):
    cell_type_list = list()
    as_name = excel_dict["AS/1"][0]
    for index in range(0, len(excel_dict["AS/1"])):
        cell_type_list.append(list())
    
    for index in range(0, len(excel_dict["AS/1"])):
        for label in key_list:
            if "CT" in label and "LABEL" not in label and "ID" not in label and "NOTES" not in label and "Reference" not in label and "DOI" not in label:
                if excel_dict[label][index]:
                    cell_type_list[index] = (excel_dict[label][index])
            if "ID" in label and "CT" in label and excel_dict[label][index]:
                try:
                    cell_id = excel_dict[label][index].replace(":", "_") 
                    url = "http://www.ebi.ac.uk/ols/api/terms?iri=http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2F{}".format(cell_id)
                    print(url)
                    cell_type = get_jdata(url)
                    cell_type_list[index] = cell_type
                except:
                    print("Fetch cell type error: Cell Ontology Not Found on EBI")
                    continue
    
    # Dont add unique identifier
    final_cell_type_list = list()
    for cell_type in cell_type_list:
        final_cell_type_list.append(cell_type)
    
    return(as_name, final_cell_type_list)

# Cell Types List
as_name, cell_type_list = add_cell_type_to_list(excel_dict, key_list)

http://www.ebi.ac.uk/ols/api/terms?iri=http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2FCL_0002071
http://www.ebi.ac.uk/ols/api/terms?iri=http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2FCL_1000321
http://www.ebi.ac.uk/ols/api/terms?iri=http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2FCL_0009016
http://www.ebi.ac.uk/ols/api/terms?iri=http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2FCL_0009009
http://www.ebi.ac.uk/ols/api/terms?iri=http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2FCL_0000165
http://www.ebi.ac.uk/ols/api/terms?iri=http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2FCL_0002071
http://www.ebi.ac.uk/ols/api/terms?iri=http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2FCL_1000321
http://www.ebi.ac.uk/ols/api/terms?iri=http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2FCL_0009011
http://www.ebi.ac.uk/ols/api/terms?iri=http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2FCL_0019032
http://www.ebi.ac.uk/ols/api/terms?iri=http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2FCL_0000915
http://www.ebi.ac.uk/ols/api/terms?iri=http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2FCL_0008364

http://www.ebi.ac.uk/ols/api/terms?iri=http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2FCL_0002138
http://www.ebi.ac.uk/ols/api/terms?iri=http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2FCL_0000669
http://www.ebi.ac.uk/ols/api/terms?iri=http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2FCL_0000057
http://www.ebi.ac.uk/ols/api/terms?iri=http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2FCL_0002573
http://www.ebi.ac.uk/ols/api/terms?iri=http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2FCL_1000412
http://www.ebi.ac.uk/ols/api/terms?iri=http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2FCL_1000414
http://www.ebi.ac.uk/ols/api/terms?iri=http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2FCL_0002138
http://www.ebi.ac.uk/ols/api/terms?iri=http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2FCL_0000136
http://www.ebi.ac.uk/ols/api/terms?iri=http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2FCL_0000669
http://www.ebi.ac.uk/ols/api/terms?iri=http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2FCL_1000457
http://www.ebi.ac.uk/ols/api/terms?iri=http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2FCL_0002071

http://www.ebi.ac.uk/ols/api/terms?iri=http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2FCL_1000412
http://www.ebi.ac.uk/ols/api/terms?iri=http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2FCL_1000414
http://www.ebi.ac.uk/ols/api/terms?iri=http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2FCL_0002138
http://www.ebi.ac.uk/ols/api/terms?iri=http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2FCL_0000669
http://www.ebi.ac.uk/ols/api/terms?iri=http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2FCL_1000279
http://www.ebi.ac.uk/ols/api/terms?iri=http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2FCL_0000057
http://www.ebi.ac.uk/ols/api/terms?iri=http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2FCL_0002573
http://www.ebi.ac.uk/ols/api/terms?iri=http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2FCL_1000412
http://www.ebi.ac.uk/ols/api/terms?iri=http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2FCL_1000414
http://www.ebi.ac.uk/ols/api/terms?iri=http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2FCL_0002138
http://www.ebi.ac.uk/ols/api/terms?iri=http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2FCL_0000669

http://www.ebi.ac.uk/ols/api/terms?iri=http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2FCL_0000057
http://www.ebi.ac.uk/ols/api/terms?iri=http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2FCL_0002573
http://www.ebi.ac.uk/ols/api/terms?iri=http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2FCL_1000412
http://www.ebi.ac.uk/ols/api/terms?iri=http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2FCL_1000414
http://www.ebi.ac.uk/ols/api/terms?iri=http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2FCL_0002138
http://www.ebi.ac.uk/ols/api/terms?iri=http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2FCL_0000669
http://www.ebi.ac.uk/ols/api/terms?iri=http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2FCL_0000669
http://www.ebi.ac.uk/ols/api/terms?iri=http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2FCL_0002573
http://www.ebi.ac.uk/ols/api/terms?iri=http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2FCL_0000057
http://www.ebi.ac.uk/ols/api/terms?iri=http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2FCL_1000412
http://www.ebi.ac.uk/ols/api/terms?iri=http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2FCL_1000414

http://www.ebi.ac.uk/ols/api/terms?iri=http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2FCL_0002652
http://www.ebi.ac.uk/ols/api/terms?iri=http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2FCL_0000669
http://www.ebi.ac.uk/ols/api/terms?iri=http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2FCL_0000682
http://www.ebi.ac.uk/ols/api/terms?iri=http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2FCL_0009018
http://www.ebi.ac.uk/ols/api/terms?iri=http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2FCL_0002573
http://www.ebi.ac.uk/ols/api/terms?iri=http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2FCL_0002573
http://www.ebi.ac.uk/ols/api/terms?iri=http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2FCL_0002573
http://www.ebi.ac.uk/ols/api/terms?iri=http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2FCL_0000669
http://www.ebi.ac.uk/ols/api/terms?iri=http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2FCL_1000279
http://www.ebi.ac.uk/ols/api/terms?iri=http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2FCL_0000057
http://www.ebi.ac.uk/ols/api/terms?iri=http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2FCL_0002573

...

http://www.ebi.ac.uk/ols/api/terms?iri=http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2FCL_1000412
http://www.ebi.ac.uk/ols/api/terms?iri=http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2FCL_1000414
http://www.ebi.ac.uk/ols/api/terms?iri=http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2FCL_0002138
http://www.ebi.ac.uk/ols/api/terms?iri=http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2FCL_0000136
http://www.ebi.ac.uk/ols/api/terms?iri=http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2FCL_0000669
http://www.ebi.ac.uk/ols/api/terms?iri=http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2FCL_0000192
http://www.ebi.ac.uk/ols/api/terms?iri=http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2FCL_0000192
http://www.ebi.ac.uk/ols/api/terms?iri=http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2FCL_0000192
http://www.ebi.ac.uk/ols/api/terms?iri=http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2FCL_1000457
http://www.ebi.ac.uk/ols/api/terms?iri=http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2FCL_0002071
http://www.ebi.ac.uk/ols/api/terms?iri=http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2FCL_1000321

http://www.ebi.ac.uk/ols/api/terms?iri=http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2FCL_1000414
http://www.ebi.ac.uk/ols/api/terms?iri=http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2FCL_0002138
http://www.ebi.ac.uk/ols/api/terms?iri=http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2FCL_0000669
http://www.ebi.ac.uk/ols/api/terms?iri=http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2FCL_1000279
http://www.ebi.ac.uk/ols/api/terms?iri=http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2FCL_0000057
http://www.ebi.ac.uk/ols/api/terms?iri=http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2FCL_0002573
http://www.ebi.ac.uk/ols/api/terms?iri=http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2FCL_1000412
http://www.ebi.ac.uk/ols/api/terms?iri=http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2FCL_1000414
http://www.ebi.ac.uk/ols/api/terms?iri=http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2FCL_0002138
http://www.ebi.ac.uk/ols/api/terms?iri=http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2FCL_0000669
http://www.ebi.ac.uk/ols/api/terms?iri=http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2FCL_0000057

http://www.ebi.ac.uk/ols/api/terms?iri=http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2FCL_0002573
http://www.ebi.ac.uk/ols/api/terms?iri=http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2FCL_1000412
http://www.ebi.ac.uk/ols/api/terms?iri=http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2FCL_1000414
http://www.ebi.ac.uk/ols/api/terms?iri=http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2FCL_0002138
http://www.ebi.ac.uk/ols/api/terms?iri=http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2FCL_0000669
http://www.ebi.ac.uk/ols/api/terms?iri=http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2FCL_0000669
http://www.ebi.ac.uk/ols/api/terms?iri=http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2FCL_0002573
http://www.ebi.ac.uk/ols/api/terms?iri=http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2FCL_0000057
http://www.ebi.ac.uk/ols/api/terms?iri=http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2FCL_1000412
http://www.ebi.ac.uk/ols/api/terms?iri=http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2FCL_1000414
http://www.ebi.ac.uk/ols/api/terms?iri=http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2FCL_0002138

http://www.ebi.ac.uk/ols/api/terms?iri=http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2FCL_0000935
http://www.ebi.ac.uk/ols/api/terms?iri=http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2FCL_0002144
http://www.ebi.ac.uk/ols/api/terms?iri=http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2FCL_0002138
http://www.ebi.ac.uk/ols/api/terms?iri=http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2FCL_0000669
http://www.ebi.ac.uk/ols/api/terms?iri=http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2FCL_0000186
http://www.ebi.ac.uk/ols/api/terms?iri=http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2FCL_0000057
http://www.ebi.ac.uk/ols/api/terms?iri=http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2FCL_0002573
http://www.ebi.ac.uk/ols/api/terms?iri=http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2FCL_0000136
http://www.ebi.ac.uk/ols/api/terms?iri=http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2FCL_0000057
http://www.ebi.ac.uk/ols/api/terms?iri=http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2FCL_0002573
http://www.ebi.ac.uk/ols/api/terms?iri=http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2FCL_1000412

http://www.ebi.ac.uk/ols/api/terms?iri=http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2FCL_0000188


## Extract biomarkers and assign to every cell type
Cell Markers is extracted depending on ASCT+B format. <br>
Case 1: A "All Gene Biomarker" column is specified in the ASCT+B Table <br>
Case 2: A "Gene Marker" column is specified in the ASCT+B Table <br>
Case 3: No biomarker column is specifed in the ASCT+B Table, however HGNC ID is given. <br>
Match HGNC ID to Genes using genenames.org API

In [6]:
def get_url(url):
    myResponse = requests.get(url)
    if (myResponse.ok):
        try:
            xml_data = myResponse.content
            soup = BeautifulSoup(xml_data, "xml")
            result_tag = soup.find("result")
            gene_tag = result_tag.find_all('str')
            print(gene_tag)
            gene_symbol = gene_tag[1].get_text()
        except:
            gene_symbol = ""

    else:
        gene_symbol = ""
    return gene_symbol

# Create a dictionary where the cell type is the key and the value is a gene marker
def add_gene_marker(cell_type_list, excel_dict):
    cell_type_gene_marker_dict = dict()
    
    for count, cell_type in enumerate(cell_type_list):
        if cell_type:
            if cell_type in cell_type_gene_marker_dict:   
                cell_type_gene_marker_dict[cell_type].update(excel_dict["All Gene Biomarkers"][count].split(", "))            
            else:
                cell_type_gene_marker_dict[cell_type] = set(excel_dict["All Gene Biomarkers"][count].split(", "))
    
    return(cell_type_gene_marker_dict)


def add_gene_protein_marker(cell_type_list, excel_dict, key_list):
    cell_type_gene_marker_dict = dict()
    
    for count, cell_type in enumerate(cell_type_list):
        if cell_type and excel_dict["Gene Markers"][count]:
            if cell_type in cell_type_gene_marker_dict:   
                cell_type_gene_marker_dict[cell_type].update(excel_dict["Gene Markers"][count].split(", "))            
            else:
                cell_type_gene_marker_dict[cell_type] = set(excel_dict["Gene Markers"][count].split(", "))

    # Add protein markers
    gene_list = list()
    for index in range(0, len(excel_dict["AS/1"])):
        gene_list.append(list())
        for label in key_list:
            if "BProtein" in label and "ID" in label and "LABEL" not in label and "Reference" not in label and "DOI" not in label:
                cell_value = excel_dict[label][index]
                cell_value_list = re.findall('HGNC:\d{4,5}', cell_value)
                cell_value_list = [i for i in cell_value_list if i]

                for cell_value in cell_value_list:
                    if cell_value:
                        url = "https://rest.genenames.org/search/hgnc_id/{}".format(cell_value.split(":")[1])
                        marker_symbol = get_url(url)
                        if marker_symbol:                    
                            gene_list[index].append(marker_symbol)

    for count, cell_type in enumerate(cell_type_list):
        if cell_type and gene_list[count]:
            if cell_type not in cell_type_gene_marker_dict.keys():
                cell_type_gene_marker_dict[cell_type] = set(gene_list[count])      
            else:
                cell_type_gene_marker_dict[cell_type].update(gene_list[count])
    
    return(cell_type_gene_marker_dict)

def add_gene_marker_no_all(cell_type_list, excel_dict, key_list):
    gene_list = list()
    for index in range(0, len(excel_dict["AS/1"])):
        gene_list.append(list())
        for label in key_list:
            cell_value = excel_dict[label][index]
            cell_value_list = re.findall('HGNC:\d{4,5}', cell_value)
            cell_value_list = [i for i in cell_value_list if i]

            for cell_value in cell_value_list:
                if cell_value:
                    url = "https://rest.genenames.org/search/hgnc_id/{}".format(cell_value.split(":")[1])
                    marker_symbol = get_url(url)
                    if marker_symbol: 
                        gene_list[index].append(marker_symbol)

    cell_type_gene_marker_dict = dict()
    for count, cell_type in enumerate(cell_type_list):
        if cell_type:
            if cell_type not in cell_type_gene_marker_dict.keys():
                cell_type_gene_marker_dict[cell_type] = set(gene_list[count])      
            else:
                cell_type_gene_marker_dict[cell_type].update(gene_list[count])
    
    return(cell_type_gene_marker_dict)


# Output of cell type/gene marker dictionary
#final_dict = add_gene_marker(cell_type_list, excel_dict, key_list)
# Output of cell type/gene marker dictionary
if "All Gene Biomarkers" in excel_dict:
    final_dict = add_gene_marker(cell_type_list, excel_dict)
elif "Gene Markers" in excel_dict:
    final_dict = add_gene_protein_marker(cell_type_list, excel_dict, key_list)
else:
    final_dict = add_gene_marker_no_all(cell_type_list, excel_dict, key_list)

https://rest.genenames.org/search/hgnc_id/20412
b'<?xml version="1.0"?>\n<response>\n\n<lst name="responseHeader">\n  <int name="status">0</int>\n  <int name="QTime">1</int>\n</lst>\n<result name="response" numFound="1" start="0" maxScore="5.8898387">\n  <doc>\n    <str name="hgnc_id">HGNC:20412</str>\n    <str name="symbol">KRT20</str>\n    <float name="score">5.8898387</float></doc>\n</result>\n</response>\n'
<result maxscore="5.8898387" name="response" numfound="1" start="0">
<doc>
<str name="hgnc_id">HGNC:20412</str>
<str name="symbol">KRT20</str>
<float name="score">5.8898387</float></doc>
</result>
https://rest.genenames.org/search/hgnc_id/17106
b'<?xml version="1.0"?>\n<response>\n\n<lst name="responseHeader">\n  <int name="status">0</int>\n  <int name="QTime">1</int>\n</lst>\n<result name="response" numFound="1" start="0" maxScore="5.8898387">\n  <doc>\n    <str name="hgnc_id">HGNC:17106</str>\n    <str name="symbol">BEST4</str>\n    <float name="score">5.8898387</float></doc>\

b'<?xml version="1.0"?>\n<response>\n\n<lst name="responseHeader">\n  <int name="status">0</int>\n  <int name="QTime">0</int>\n</lst>\n<result name="response" numFound="1" start="0" maxScore="5.8898387">\n  <doc>\n    <str name="hgnc_id">HGNC:26896</str>\n    <str name="symbol">RGMB</str>\n    <float name="score">5.8898387</float></doc>\n</result>\n</response>\n'
<result maxscore="5.8898387" name="response" numfound="1" start="0">
<doc>
<str name="hgnc_id">HGNC:26896</str>
<str name="symbol">RGMB</str>
<float name="score">5.8898387</float></doc>
</result>
https://rest.genenames.org/search/hgnc_id/11204
b'<?xml version="1.0"?>\n<response>\n\n<lst name="responseHeader">\n  <int name="status">0</int>\n  <int name="QTime">0</int>\n</lst>\n<result name="response" numFound="1" start="0" maxScore="5.8898387">\n  <doc>\n    <str name="hgnc_id">HGNC:11204</str>\n    <str name="symbol">SOX9</str>\n    <float name="score">5.8898387</float></doc>\n</result>\n</response>\n'
<result maxscore="5.8898

b'<?xml version="1.0"?>\n<response>\n\n<lst name="responseHeader">\n  <int name="status">0</int>\n  <int name="QTime">0</int>\n</lst>\n<result name="response" numFound="1" start="0" maxScore="5.8898387">\n  <doc>\n    <str name="hgnc_id">HGNC:6090</str>\n    <str name="symbol">INSM1</str>\n    <float name="score">5.8898387</float></doc>\n</result>\n</response>\n'
<result maxscore="5.8898387" name="response" numfound="1" start="0">
<doc>
<str name="hgnc_id">HGNC:6090</str>
<str name="symbol">INSM1</str>
<float name="score">5.8898387</float></doc>
</result>
https://rest.genenames.org/search/hgnc_id/6088
b'<?xml version="1.0"?>\n<response>\n\n<lst name="responseHeader">\n  <int name="status">0</int>\n  <int name="QTime">0</int>\n</lst>\n<result name="response" numFound="1" start="0" maxScore="5.8898387">\n  <doc>\n    <str name="hgnc_id">HGNC:6088</str>\n    <str name="symbol">INSL5</str>\n    <float name="score">5.8898387</float></doc>\n</result>\n</response>\n'
<result maxscore="5.88983

b'<?xml version="1.0"?>\n<response>\n\n<lst name="responseHeader">\n  <int name="status">0</int>\n  <int name="QTime">2</int>\n</lst>\n<result name="response" numFound="1" start="0" maxScore="5.8898387">\n  <doc>\n    <str name="hgnc_id">HGNC:14323</str>\n    <str name="symbol">TRPM5</str>\n    <float name="score">5.8898387</float></doc>\n</result>\n</response>\n'
<result maxscore="5.8898387" name="response" numfound="1" start="0">
<doc>
<str name="hgnc_id">HGNC:14323</str>
<str name="symbol">TRPM5</str>
<float name="score">5.8898387</float></doc>
</result>
https://rest.genenames.org/search/hgnc_id/19864
b'<?xml version="1.0"?>\n<response>\n\n<lst name="responseHeader">\n  <int name="status">0</int>\n  <int name="QTime">0</int>\n</lst>\n<result name="response" numFound="1" start="0" maxScore="5.8898387">\n  <doc>\n    <str name="hgnc_id">HGNC:19864</str>\n    <str name="symbol">POU2F3</str>\n    <float name="score">5.8898387</float></doc>\n</result>\n</response>\n'
<result maxscore="5.

b'<?xml version="1.0"?>\n<response>\n\n<lst name="responseHeader">\n  <int name="status">0</int>\n  <int name="QTime">0</int>\n</lst>\n<result name="response" numFound="1" start="0" maxScore="5.8898387">\n  <doc>\n    <str name="hgnc_id">HGNC:1676</str>\n    <str name="symbol">CD3W</str>\n    <float name="score">5.8898387</float></doc>\n</result>\n</response>\n'
<result maxscore="5.8898387" name="response" numfound="1" start="0">
<doc>
<str name="hgnc_id">HGNC:1676</str>
<str name="symbol">CD3W</str>
<float name="score">5.8898387</float></doc>
</result>
https://rest.genenames.org/search/hgnc_id/1695
b'<?xml version="1.0"?>\n<response>\n\n<lst name="responseHeader">\n  <int name="status">0</int>\n  <int name="QTime">0</int>\n</lst>\n<result name="response" numFound="1" start="0" maxScore="5.8898387">\n  <doc>\n    <str name="hgnc_id">HGNC:1695</str>\n    <str name="symbol">CD7</str>\n    <float name="score">5.8898387</float></doc>\n</result>\n</response>\n'
<result maxscore="5.8898387" 

...

b'<?xml version="1.0"?>\n<response>\n\n<lst name="responseHeader">\n  <int name="status">0</int>\n  <int name="QTime">0</int>\n</lst>\n<result name="response" numFound="1" start="0" maxScore="5.8898387">\n  <doc>\n    <str name="hgnc_id">HGNC:8823</str>\n    <str name="symbol">PECAM1</str>\n    <float name="score">5.8898387</float></doc>\n</result>\n</response>\n'
<result maxscore="5.8898387" name="response" numfound="1" start="0">
<doc>
<str name="hgnc_id">HGNC:8823</str>
<str name="symbol">PECAM1</str>
<float name="score">5.8898387</float></doc>
</result>
https://rest.genenames.org/search/hgnc_id/1662
b'<?xml version="1.0"?>\n<response>\n\n<lst name="responseHeader">\n  <int name="status">0</int>\n  <int name="QTime">0</int>\n</lst>\n<result name="response" numFound="1" start="0" maxScore="5.8898387">\n  <doc>\n    <str name="hgnc_id">HGNC:1662</str>\n    <str name="symbol">CD34</str>\n    <float name="score">5.8898387</float></doc>\n</result>\n</response>\n'
<result maxscore="5.8898

b'<?xml version="1.0"?>\n<response>\n\n<lst name="responseHeader">\n  <int name="status">0</int>\n  <int name="QTime">0</int>\n</lst>\n<result name="response" numFound="1" start="0" maxScore="5.8898387">\n  <doc>\n    <str name="hgnc_id">HGNC:2211</str>\n    <str name="symbol">COL6A1</str>\n    <float name="score">5.8898387</float></doc>\n</result>\n</response>\n'
<result maxscore="5.8898387" name="response" numfound="1" start="0">
<doc>
<str name="hgnc_id">HGNC:2211</str>
<str name="symbol">COL6A1</str>
<float name="score">5.8898387</float></doc>
</result>
https://rest.genenames.org/search/hgnc_id/2212
b'<?xml version="1.0"?>\n<response>\n\n<lst name="responseHeader">\n  <int name="status">0</int>\n  <int name="QTime">0</int>\n</lst>\n<result name="response" numFound="1" start="0" maxScore="5.8898387">\n  <doc>\n    <str name="hgnc_id">HGNC:2212</str>\n    <str name="symbol">COL6A2</str>\n    <float name="score">5.8898387</float></doc>\n</result>\n</response>\n'
<result maxscore="5.88

b'<?xml version="1.0"?>\n<response>\n\n<lst name="responseHeader">\n  <int name="status">0</int>\n  <int name="QTime">0</int>\n</lst>\n<result name="response" numFound="1" start="0" maxScore="5.8898387">\n  <doc>\n    <str name="hgnc_id">HGNC:1662</str>\n    <str name="symbol">CD34</str>\n    <float name="score">5.8898387</float></doc>\n</result>\n</response>\n'
<result maxscore="5.8898387" name="response" numfound="1" start="0">
<doc>
<str name="hgnc_id">HGNC:1662</str>
<str name="symbol">CD34</str>
<float name="score">5.8898387</float></doc>
</result>
https://rest.genenames.org/search/hgnc_id/1663
b'<?xml version="1.0"?>\n<response>\n\n<lst name="responseHeader">\n  <int name="status">0</int>\n  <int name="QTime">0</int>\n</lst>\n<result name="response" numFound="1" start="0" maxScore="5.8898387">\n  <doc>\n    <str name="hgnc_id">HGNC:1663</str>\n    <str name="symbol">CD36</str>\n    <float name="score">5.8898387</float></doc>\n</result>\n</response>\n'
<result maxscore="5.8898387"

b'<?xml version="1.0"?>\n<response>\n\n<lst name="responseHeader">\n  <int name="status">0</int>\n  <int name="QTime">0</int>\n</lst>\n<result name="response" numFound="1" start="0" maxScore="5.8898387">\n  <doc>\n    <str name="hgnc_id">HGNC:1676</str>\n    <str name="symbol">CD3W</str>\n    <float name="score">5.8898387</float></doc>\n</result>\n</response>\n'
<result maxscore="5.8898387" name="response" numfound="1" start="0">
<doc>
<str name="hgnc_id">HGNC:1676</str>
<str name="symbol">CD3W</str>
<float name="score">5.8898387</float></doc>
</result>
https://rest.genenames.org/search/hgnc_id/1695
b'<?xml version="1.0"?>\n<response>\n\n<lst name="responseHeader">\n  <int name="status">0</int>\n  <int name="QTime">0</int>\n</lst>\n<result name="response" numFound="1" start="0" maxScore="5.8898387">\n  <doc>\n    <str name="hgnc_id">HGNC:1695</str>\n    <str name="symbol">CD7</str>\n    <float name="score">5.8898387</float></doc>\n</result>\n</response>\n'
<result maxscore="5.8898387" 

ConnectionError: ('Connection aborted.', ConnectionResetError(54, 'Connection reset by peer'))

## Create output directory
Create an output directory if it does not exit already and change current directory to output directory

In [ ]:
# Check if output directory exist
output_path = '{}/outdir_3'.format(os.getcwd())
if os.path.isdir(output_path):
    # Change the current working directory to output directory
    os.chdir(output_path)
else:
    # Create output directory if it does not exist
    os.mkdir(output_path)
    os.chdir(output_path)

## Generate GMT files

In [ ]:
output_filename = "{}_GMT_{}.tsv".format(file_name.split("/")[-1][:-4], version)
fh = open(output_filename , "w")
for key, value in final_dict.items():
    key = key.strip('\t') + ':' + as_name.title()
    if "[" in key or not value:
        continue
    elif value and type(value) == list:
        fh.write("{}\t{}\n".format(key.strip('\t'), ';'.join(value)))
    elif value:
        fh.write("{}\t{}\n".format(key, ';'.join(value)))
    else:
        fh.write("{}\n".format(key))